# US Golf Course Information Kiosk

Detailed information on golf courses with green fees range and tees, this complete golf courses database has 15,606 records across 7,891 cities over 53 states in the United States. Each record is comprised of address, street, phone number, zip code, hole, architect, year built, public/private, guest policy, credit card, golf season, range, rental club, pro in house, metal spikes okay, weekday, weekend, tee time welcomed, rental cart available, championship par/yards/slope/USGA, middle par/yards/slope/USGA and forward par/yards/slope/USGA

https://www.usabledatabases.com/database/golf-courses-in-us/

## Step 1:  Clean and transform the input dataset

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import json
from sqlalchemy import create_engine
from config import db_connection

# Import API key
from config import gkey

# CSV files
course = "data/course.csv"
city = "data/city.csv"
state = "data/state.csv"

# Read the golf course dataset
input_csv = pd.read_csv("data/course.csv", delimiter=',', skipinitialspace=True)
input_csv.shape

(15606, 34)

In [2]:
# Create the data frame - golf courses
course_df = pd.read_csv(course, delimiter=',', skipinitialspace=True)

# Add columns for longitude and latitude
course_df["lng"] = ""
course_df["lat"] = ""

golf_courses = course_df.rename(columns={"id": "course_id", 
                                        "title": "course"})
golf_courses

,course_id,city_id,course,street,address,zip_code,phone,hole,architect,year_built,...,middle_par,middle_yards,middle_slope,middle_usga,forward_par,forward_yards,forward_slope,forward_usga,lng,lat
0,1,1,Albertville Golf & Country Club,Country Club Rd,"Albertville, Alabama 35950",35950.0,(256) 878-4403,18.0,Leon Howard,1966,...,72,6068,117,68.00,72,5196,NaN,NaN,,
1,2,1,A. J. Jolly Golf Course,5350 S US Hwy 27,"Alexandria, Kentucky 41001",41001.0,(606) 635-2106,18.0,NaN,1962,...,71,5942,115,67.60,75,5418,118,70.30,,
2,3,2,Willow Point Country Club,3054 Willow Point Rd,"Alexander City, Alabama 35010",35010.0,(256) 234-2572,18.0,Thomas H. Nicol,1961,...,72,6631,135,72.20,73,5373,120,70.70,,
3,4,3,Alpine Bay Resort,9855 Renfroe Rd,"Alpine, Alabama 35014",35014.0,(256) 268-2920,18.0,"Robert Trent Jones, Sr.",1972,...,72,6518,126,69.90,72,5518,120,69.80,,
4,5,4,Maple Hills Golf,Hwy 75,"Altoona, Alabama 35952",35952.0,(205) 466-7600,9.0,Bill Ellison,1975,...,31,1800,NaN,NaN,31,1800,NaN,NaN,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15601,15750,8219,Legion Town & Country Club,141 Airport Rd,"Thermopolis, Wyoming 82443",82443.0,(307) 864-5294,9.0,Don Brunk,1962,...,35,2966,111,33.90,37,2836,111,35.00,,
15602,15751,8220,Cottonwood Country Club,15th St,"Torrington, Wyoming 82240",82240.0,(307) 532-3868,18.0,NaN,NaN,...,72,6298,112,67.70,73,5344,112,69.20,,
15603,15752,8221,Cedar Pines Golf Course,2579 N Hwy 116,"Upton, Wyoming 82730",82730.0,(307) 468-2847,9.0,NaN,1984,...,36,3198,NaN,NaN,38,2521,NaN,NaN,,
15604,15753,8222,Wheatland Golf Club,1253 E Cole,"Wheatland, Wyoming 82201",82201.0,(307) 322-3675,9.0,NaN,1960,...,36,3064,119,33.95,36,2557,116,34.25,,


In [3]:
# Checking for duplicate rows in golf courses
duplicateRow = golf_courses[golf_courses.duplicated()]
duplicateRow

,course_id,city_id,course,street,address,zip_code,phone,hole,architect,year_built,...,middle_par,middle_yards,middle_slope,middle_usga,forward_par,forward_yards,forward_slope,forward_usga,lng,lat


In [4]:
# Create the data frame - golf cities
city_df = pd.read_csv(city, delimiter=',', skipinitialspace=True)

golf_cities = city_df.rename(columns={"id": "city_id", 
                                      "title": "city", 
                                      "count": "city_count", 
                                      "slug": "city_slug"})
golf_cities

,city_id,state_id,city,city_count,city_slug
0,1,1,Albertville,2,albertville
1,2,1,Alexander City,1,alexandercity
2,3,1,Alpine,1,alpine
3,4,1,Altoona,1,altoona
4,5,1,Andalusia,2,andalusia
...,...,...,...,...,...
7886,8219,53,Thermopolis,1,thermopolis
7887,8220,53,Torrington,1,torrington
7888,8221,53,Upton,1,upton
7889,8222,53,Wheatland,1,wheatland


In [5]:
# Checking for duplicate rows in golf cities
duplicateRow = golf_cities[golf_cities.duplicated()]
duplicateRow

,city_id,state_id,city,city_count,city_slug


In [6]:
# Create the data frame - golf states
state_df = pd.read_csv(state, delimiter=',', skipinitialspace=True)

golf_states = state_df.rename(columns={"id": "state_id", 
                                       "title": "state", 
                                       "count": "state_count", 
                                       "slug": "state_slug"})
golf_states

,state_id,state,state_count,state_slug
0,1,Alabama,272,alabama
1,2,Alaska,16,alaska
2,3,Arizona,308,arizona
3,4,Arkansas,179,arkansas
4,5,California,976,california
5,6,Colorado,223,colorado
6,7,Connecticut,177,connecticut
7,8,Delaware,39,delaware
8,9,District of Columbia,6,districtofcolumbia
9,10,Florida,1094,florida


In [7]:
# Checking for duplicate rows in golf states
duplicateRow = golf_states[golf_states.duplicated()]
duplicateRow

,state_id,state,state_count,state_slug


### Merge the data frames to create one complete dataset

In [8]:
# Merge the city and state dataframes using a left join
city_state_df = pd.merge(golf_cities, golf_states, on="state_id", how="left")
city_state_df

,city_id,state_id,city,city_count,city_slug,state,state_count,state_slug
0,1,1,Albertville,2,albertville,Alabama,272,alabama
1,2,1,Alexander City,1,alexandercity,Alabama,272,alabama
2,3,1,Alpine,1,alpine,Alabama,272,alabama
3,4,1,Altoona,1,altoona,Alabama,272,alabama
4,5,1,Andalusia,2,andalusia,Alabama,272,alabama
...,...,...,...,...,...,...,...,...
7886,8219,53,Thermopolis,1,thermopolis,Wyoming,48,wyoming
7887,8220,53,Torrington,1,torrington,Wyoming,48,wyoming
7888,8221,53,Upton,1,upton,Wyoming,48,wyoming
7889,8222,53,Wheatland,1,wheatland,Wyoming,48,wyoming


In [9]:
# Merge the city_state dataframe with the golf course dataframe using a left join
golf_courses = pd.merge(golf_courses, city_state_df, on="city_id" ,how="left")
golf_courses

,course_id,city_id,course,street,address,zip_code,phone,hole,architect,year_built,...,forward_usga,lng,lat,state_id,city,city_count,city_slug,state,state_count,state_slug
0,1,1,Albertville Golf & Country Club,Country Club Rd,"Albertville, Alabama 35950",35950.0,(256) 878-4403,18.0,Leon Howard,1966,...,NaN,,,1,Albertville,2,albertville,Alabama,272,alabama
1,2,1,A. J. Jolly Golf Course,5350 S US Hwy 27,"Alexandria, Kentucky 41001",41001.0,(606) 635-2106,18.0,NaN,1962,...,70.30,,,1,Albertville,2,albertville,Alabama,272,alabama
2,3,2,Willow Point Country Club,3054 Willow Point Rd,"Alexander City, Alabama 35010",35010.0,(256) 234-2572,18.0,Thomas H. Nicol,1961,...,70.70,,,1,Alexander City,1,alexandercity,Alabama,272,alabama
3,4,3,Alpine Bay Resort,9855 Renfroe Rd,"Alpine, Alabama 35014",35014.0,(256) 268-2920,18.0,"Robert Trent Jones, Sr.",1972,...,69.80,,,1,Alpine,1,alpine,Alabama,272,alabama
4,5,4,Maple Hills Golf,Hwy 75,"Altoona, Alabama 35952",35952.0,(205) 466-7600,9.0,Bill Ellison,1975,...,NaN,,,1,Altoona,1,altoona,Alabama,272,alabama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15601,15750,8219,Legion Town & Country Club,141 Airport Rd,"Thermopolis, Wyoming 82443",82443.0,(307) 864-5294,9.0,Don Brunk,1962,...,35.00,,,53,Thermopolis,1,thermopolis,Wyoming,48,wyoming
15602,15751,8220,Cottonwood Country Club,15th St,"Torrington, Wyoming 82240",82240.0,(307) 532-3868,18.0,NaN,NaN,...,69.20,,,53,Torrington,1,torrington,Wyoming,48,wyoming
15603,15752,8221,Cedar Pines Golf Course,2579 N Hwy 116,"Upton, Wyoming 82730",82730.0,(307) 468-2847,9.0,NaN,1984,...,NaN,,,53,Upton,1,upton,Wyoming,48,wyoming
15604,15753,8222,Wheatland Golf Club,1253 E Cole,"Wheatland, Wyoming 82201",82201.0,(307) 322-3675,9.0,NaN,1960,...,34.25,,,53,Wheatland,1,wheatland,Wyoming,48,wyoming


In [10]:
# Select the golf course columns needed for course longitude, latitude
golf_course_df = golf_courses[["course_id", 
                               "course", 
                               "city", 
                               "state", 
                               "street", 
                               "zip_code", 
                               "lng", 
                               "lat", 
                               "hole"]]
                     
# Display the new dataframe
golf_course_df

,course_id,course,city,state,street,zip_code,lng,lat,hole
0,1,Albertville Golf & Country Club,Albertville,Alabama,Country Club Rd,35950.0,,,18.0
1,2,A. J. Jolly Golf Course,Albertville,Alabama,5350 S US Hwy 27,41001.0,,,18.0
2,3,Willow Point Country Club,Alexander City,Alabama,3054 Willow Point Rd,35010.0,,,18.0
3,4,Alpine Bay Resort,Alpine,Alabama,9855 Renfroe Rd,35014.0,,,18.0
4,5,Maple Hills Golf,Altoona,Alabama,Hwy 75,35952.0,,,9.0
...,...,...,...,...,...,...,...,...,...
15601,15750,Legion Town & Country Club,Thermopolis,Wyoming,141 Airport Rd,82443.0,,,9.0
15602,15751,Cottonwood Country Club,Torrington,Wyoming,15th St,82240.0,,,18.0
15603,15752,Cedar Pines Golf Course,Upton,Wyoming,2579 N Hwy 116,82730.0,,,9.0
15604,15753,Wheatland Golf Club,Wheatland,Wyoming,1253 E Cole,82201.0,,,9.0


In [11]:
# Select the golf course columns needed for golf kiosk - general information
course_info = golf_courses[["course_id", 
                               "phone", 
                               "hole", 
                               "architect", 
                               "year_built", 
                               "public_private", 
                               "guest_policy", 
                               "credit_card"]]
                     
# Display the new dataframe
course_info

,course_id,phone,hole,architect,year_built,public_private,guest_policy,credit_card
0,1,(256) 878-4403,18.0,Leon Howard,1966,Private,Closed,None
1,2,(606) 635-2106,18.0,NaN,1962,Public,Open,"VISA, MasterCard Welcomed"
2,3,(256) 234-2572,18.0,Thomas H. Nicol,1961,Private,Closed,"VISA, MasterCard, Amex Accepted"
3,4,(256) 268-2920,18.0,"Robert Trent Jones, Sr.",1972,Resort,Open,"VISA, MasterCard, Amex, Discover Accepted"
4,5,(205) 466-7600,9.0,Bill Ellison,1975,Public,Open,None
...,...,...,...,...,...,...,...,...
15601,15750,(307) 864-5294,9.0,Don Brunk,1962,Public,Open,"VISA, MasterCard, Amex Welcomed"
15602,15751,(307) 532-3868,18.0,NaN,NaN,Public,Open,"VISA, MasterCard Welcomed"
15603,15752,(307) 468-2847,9.0,NaN,1984,Semi-Private,Open,None
15604,15753,(307) 322-3675,9.0,NaN,1960,Public,Open,"VISA, MasterCard, Amex, Discover Welcomed"


In [12]:
# Select the golf course columns needed for golf kiosk - course details
course_details = golf_courses[["course_id", 
                               "golf_season",
                               "range", 
                               "rental_club", 
                               "pro_in_House", 
                               "metal_spikes_okay", 
                               "weekday", 
                               "weekend", 
                               "tee_time_welcomed", 
                               "rental_cart_available"]]
                     
# Display the new dataframe
course_details

,course_id,golf_season,range,rental_club,pro_in_House,metal_spikes_okay,weekday,weekend,tee_time_welcomed,rental_cart_available
0,1,year round,No,No,No,Yes,$,$,No,No
1,2,year round,No,Yes,Yes,Yes,$,$,Yes,Yes
2,3,year round,Yes,Yes,Yes,Yes,$,$,Yes,Yes
3,4,year round,Yes,Yes,Yes,No,$,$,Yes,Yes
4,5,year round,No,No,Yes,Yes,$,$,No,No
...,...,...,...,...,...,...,...,...,...,...
15601,15750,Open: 4/01 Closed: 11/01,Yes,Yes,Yes,No,$,$,Yes,Yes
15602,15751,Open: 4/01 Closed: 11/01,Yes,Yes,No,No,$,$,No,Yes
15603,15752,Open: 4/01 Closed: 11/01,Yes,Yes,No,Yes,$,$,No,Yes
15604,15753,Open: 4/01 Closed: 11/01,Yes,Yes,No,No,$,$,Yes,Yes


In [13]:
# Select the golf course columns needed for golf kiosk - championship tees
championship_tees = golf_courses[["course_id", 
                            "championship_par", 
                            "championship_yards", 
                            "championship_slope", 
                            "championship_usga"]]
                     
# Display the new dataframe
championship_tees

,course_id,championship_par,championship_yards,championship_slope,championship_usga
0,1,72,6332,118,68.80
1,2,71,6219,118,69.30
2,3,72,7084,137,74.60
3,4,72,6900,129,70.90
4,5,NaN,NaN,NaN,NaN
...,...,...,...,...,...
15601,15750,NaN,NaN,NaN,NaN
15602,15751,NaN,NaN,NaN,NaN
15603,15752,NaN,NaN,NaN,NaN
15604,15753,NaN,NaN,NaN,NaN


In [14]:
# Select the golf course columns needed for golf kiosk - middle tees
middle_tees = golf_courses[["course_id", 
                                  "middle_par", 
                                  "middle_yards", 
                                  "middle_slope", 
                                  "middle_usga"]]
                     
# Display the new dataframe
middle_tees

,course_id,middle_par,middle_yards,middle_slope,middle_usga
0,1,72,6068,117,68.00
1,2,71,5942,115,67.60
2,3,72,6631,135,72.20
3,4,72,6518,126,69.90
4,5,31,1800,NaN,NaN
...,...,...,...,...,...
15601,15750,35,2966,111,33.90
15602,15751,72,6298,112,67.70
15603,15752,36,3198,NaN,NaN
15604,15753,36,3064,119,33.95


In [15]:
# Select the golf course columns needed for golf kiosk - forward tees
forward_tees = golf_courses[["course_id", 
                                  "forward_par", 
                                  "forward_yards", 
                                  "forward_slope", 
                                  "forward_usga"]]
                     
# Display the new dataframe
forward_tees

,course_id,forward_par,forward_yards,forward_slope,forward_usga
0,1,72,5196,NaN,NaN
1,2,75,5418,118,70.30
2,3,73,5373,120,70.70
3,4,72,5518,120,69.80
4,5,31,1800,NaN,NaN
...,...,...,...,...,...
15601,15750,37,2836,111,35.00
15602,15751,73,5344,112,69.20
15603,15752,38,2521,NaN,NaN
15604,15753,36,2557,116,34.25


In [16]:
# Create subst of golf courses
course_subset = golf_course_df[((golf_course_df["zip_code"] >= 55001) & (golf_course_df["zip_code"] <= 56763))]
course_subset

,course_id,course,city,state,street,zip_code,lng,lat,hole
7584,7671,Heart of the Valley Golf Club,Ada,Minnesota,SE Corner of Ada,56510.0,,,9.0
7585,7672,Cedar River Country Club,Adams,Minnesota,Hwy 56 W,55909.0,,,18.0
7586,7673,Adrian Golf Course,Adrian,Minnesota,Hwy 91 S,56110.0,,,9.0
7587,7674,Albany Golf Club,Albany,Minnesota,500 Church St,56307.0,,,18.0
7588,7675,Albert Lea Golf Club,Albert Lea,Minnesota,1701 Country Club Rd,56007.0,,,18.0
...,...,...,...,...,...,...,...,...,...
8063,8155,Prairie View Golf Links,Worthington,Minnesota,Hwy 266 No,56187.0,,,18.0
8064,8156,Worthington Country Club,Worthington,Minnesota,1414 Liberty Dr,56187.0,,,18.0
8065,8157,"Greenwood Golf Links, Inc.",Wyoming,Minnesota,4520 E Viking Blvd,55092.0,,,18.0
8066,8158,Fairway Shores Golf Course,Zimmerman,Minnesota,28340 131St St,55398.0,,,9.0


In [17]:
# Getting the longitude and latitude for each golf course
count = 0

# Use the street, city, state to get lng and lat
for index, row in course_subset.iterrows():
    # get street, city, state from golf_course_df
    street = row["street"]
    city = row["city"]
    state = row["state"]

    # Getting lat/lng
    query_address_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={street},+{city},+{state}&key={gkey}"
    # print(f"url = {query_address_url}")

    response = requests.get(query_address_url).json()
    # print(f"response = {response}")
    
    count = count + 1
    # print(f"#{count} - index = {index}")
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        course_subset.loc[index, "lat"] = response["results"][0]["geometry"]["location"]["lat"]
        course_subset.loc[index, "lng"] = response["results"][0]["geometry"]["location"]["lng"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

C:\Users\Public\anaconda3\envs\NewPythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Missing field/result... skipping.


In [18]:
# Preview course subset
course_subset

,course_id,course,city,state,street,zip_code,lng,lat,hole
7584,7671,Heart of the Valley Golf Club,Ada,Minnesota,SE Corner of Ada,56510.0,-96.5153,47.2997,9.0
7585,7672,Cedar River Country Club,Adams,Minnesota,Hwy 56 W,55909.0,-92.7193,43.5654,18.0
7586,7673,Adrian Golf Course,Adrian,Minnesota,Hwy 91 S,56110.0,-95.9331,43.635,9.0
7587,7674,Albany Golf Club,Albany,Minnesota,500 Church St,56307.0,-94.5703,45.6291,18.0
7588,7675,Albert Lea Golf Club,Albert Lea,Minnesota,1701 Country Club Rd,56007.0,-93.3907,43.6706,18.0
...,...,...,...,...,...,...,...,...,...
8063,8155,Prairie View Golf Links,Worthington,Minnesota,Hwy 266 No,56187.0,-95.7137,43.7012,18.0
8064,8156,Worthington Country Club,Worthington,Minnesota,1414 Liberty Dr,56187.0,-95.6228,43.6255,18.0
8065,8157,"Greenwood Golf Links, Inc.",Wyoming,Minnesota,4520 E Viking Blvd,55092.0,-93.0147,45.3406,18.0
8066,8158,Fairway Shores Golf Course,Zimmerman,Minnesota,28340 131St St,55398.0,-93.6016,45.4848,9.0


In [20]:
# Export the golf course data into a csv
course_subset.to_csv("results/course_subset.csv", index=False, header=True)